In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

from sklearn import linear_model

from sklearn.model_selection import cross_val_score
import sklearn.metrics as met

In [22]:
train = pd.read_csv("dados_lab_classificacao/train.csv")
test = pd.read_csv("dados_lab_classificacao/test.csv")

In [23]:
train = train.drop(['sequencial_candidato','ano','nome','cargo', 'grau','ocupacao', 'estado_civil', 'partido', 'sexo'], axis=1)
train = train.drop(['quantidade_doadores', 'total_receita','recursos_de_pessoas_juridicas', 'quantidade_despesas', 'quantidade_fornecedores'], axis=1)
train.head()

,uf,quantidade_doacoes,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_proprios,recursos_de_partido_politico,total_despesa,media_despesa,situacao
0,AP,6,2766.67,0.0,9000.0,1300.0,0.0,16583.60,1184.54,nao_eleito
1,RO,13,1755.85,6625.0,15000.0,201.0,0.0,20325.99,846.92,nao_eleito
2,AP,17,9301.22,2250.0,34150.0,59500.0,0.0,146011.70,1187.09,eleito
3,MS,6,500.19,0.0,1150.0,750.0,0.0,3001.12,375.14,nao_eleito
4,RO,48,2496.25,0.0,50878.0,68942.0,0.0,116416.64,875.31,eleito


In [24]:
test_seq_cand = test['sequencial_candidato']
test = test.drop(['sequencial_candidato', 'ano','nome', 'cargo', 'grau','ocupacao', 'estado_civil', 'partido', 'sexo'], axis=1)
test = test.drop(['quantidade_doadores', 'total_receita','recursos_de_pessoas_juridicas', 'quantidade_despesas', 'quantidade_fornecedores'], axis=1)
test.head()

,uf,quantidade_doacoes,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_proprios,recursos_de_partido_politico,total_despesa,media_despesa
0,AC,3,526.67,0.00,1500.00,0.00,80.0,1580.00,526.67
1,AC,5,636.00,0.00,3100.00,0.00,80.0,3130.02,521.67
2,AC,40,8419.83,1923.07,65700.00,115000.00,0.0,326869.78,2254.27
3,AC,29,5404.11,39122.32,68783.84,6813.16,25000.0,241016.07,1772.18
4,AC,160,4606.71,10000.00,347073.00,78000.00,25000.0,567401.15,1095.37


In [25]:
train_X = train.loc[:,'uf':'media_despesa']
train_Y =  train['situacao']
train_Y.head()

0    nao_eleito
1    nao_eleito
2        eleito
3    nao_eleito
4        eleito
Name: situacao, dtype: object

In [26]:
#log transform skewed numeric features (train):
numeric_feats = train_X.dtypes[train_X.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x)) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

train_X[skewed_feats] = np.log1p(train_X[skewed_feats])

#dummyfying
train_X = pd.get_dummies(train_X)

#log transform skewed numeric features (test):
numeric_feats = test.dtypes[test.dtypes != "object"].index

skewed_feats = test[numeric_feats].apply(lambda x: skew(x)) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

test[skewed_feats] = np.log1p(test[skewed_feats])

#dummyfying
test = pd.get_dummies(test)

In [27]:
train_X.head()

,quantidade_doacoes,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_proprios,recursos_de_partido_politico,total_despesa,media_despesa,uf_AC,uf_AL,...,uf_PR,uf_RJ,uf_RN,uf_RO,uf_RR,uf_RS,uf_SC,uf_SE,uf_SP,uf_TO
0,1.945910,7.925761,0.000000,9.105091,7.170888,0.0,9.716230,7.077954,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.639057,7.471278,8.798757,9.615872,5.308268,0.0,9.919705,6.742786,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2.890372,9.138008,7.719130,10.438547,10.993748,0.0,11.891449,7.080102,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.945910,6.216985,0.000000,7.048386,6.621406,0.0,8.007074,5.929961,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.891820,7.822945,0.000000,10.837206,11.141035,0.0,11.664939,6.775720,0,0,...,0,0,0,1,0,0,0,0,0,0


In [28]:
test.head()

,quantidade_doacoes,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_proprios,recursos_de_partido_politico,total_despesa,media_despesa,uf_AC,uf_AL,...,uf_PR,uf_RJ,uf_RN,uf_RO,uf_RR,uf_RS,uf_SC,uf_SE,uf_SP,uf_TO
0,1.386294,6.268471,0.000000,7.313887,0.000000,4.394449,7.365813,6.268471,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1.791759,6.456770,0.000000,8.039480,0.000000,4.394449,8.049114,6.258950,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3.713572,9.038464,7.562198,11.092869,11.652696,0.000000,12.697320,7.721025,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3.401197,8.595100,10.574474,11.138739,8.826758,10.126671,12.392623,7.480530,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5.081404,8.435486,9.210440,12.757293,11.264477,10.126671,13.248824,6.999760,1,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
from sklearn import preprocessing
#creating labelEncoder
leTarget = preprocessing.LabelEncoder()
classes = ['nao_eleito', 'eleito']
leTarget.fit(classes)
print(leTarget.classes_)
# Converting string labels into numbers.
#nao eleito to 1, eleito to 0
train_Y=leTarget.fit_transform(train_Y)
print(train_Y)

['eleito' 'nao_eleito']
[1 1 0 ... 1 1 1]


In [31]:
train_X.head()

,quantidade_doacoes,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_proprios,recursos_de_partido_politico,total_despesa,media_despesa,uf_AC,uf_AL,...,uf_PR,uf_RJ,uf_RN,uf_RO,uf_RR,uf_RS,uf_SC,uf_SE,uf_SP,uf_TO
0,1.945910,7.925761,0.000000,9.105091,7.170888,0.0,9.716230,7.077954,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.639057,7.471278,8.798757,9.615872,5.308268,0.0,9.919705,6.742786,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2.890372,9.138008,7.719130,10.438547,10.993748,0.0,11.891449,7.080102,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.945910,6.216985,0.000000,7.048386,6.621406,0.0,8.007074,5.929961,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.891820,7.822945,0.000000,10.837206,11.141035,0.0,11.664939,6.775720,0,0,...,0,0,0,1,0,0,0,0,0,0


In [32]:
test.head()

,quantidade_doacoes,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_proprios,recursos_de_partido_politico,total_despesa,media_despesa,uf_AC,uf_AL,...,uf_PR,uf_RJ,uf_RN,uf_RO,uf_RR,uf_RS,uf_SC,uf_SE,uf_SP,uf_TO
0,1.386294,6.268471,0.000000,7.313887,0.000000,4.394449,7.365813,6.268471,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1.791759,6.456770,0.000000,8.039480,0.000000,4.394449,8.049114,6.258950,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3.713572,9.038464,7.562198,11.092869,11.652696,0.000000,12.697320,7.721025,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3.401197,8.595100,10.574474,11.138739,8.826758,10.126671,12.392623,7.480530,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5.081404,8.435486,9.210440,12.757293,11.264477,10.126671,13.248824,6.999760,1,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(cv=10, random_state=0)
lr.fit(train_X, train_Y)
train_y_pred = lr.predict(train_X)

train_precision = met.precision_score(train_Y, train_y_pred)
train_recall = met.recall_score(train_Y, train_y_pred)
train_accuracy = met.accuracy_score(train_Y, train_y_pred)
precisions, recalls, thresholds = met.precision_recall_curve(train_Y, lr.predict_proba(train_X)[:,1])
train_area = met.auc(recalls, precisions)

print("Train")
print("Acuracy: ", train_accuracy)
print("Precision: ", train_precision)
print("Recall: ", train_recall)
print("AUC - P&R: ", train_area)

y_pred = lr.predict(test)
y_pred = leTarget.inverse_transform(y_pred)
result = pd.DataFrame()
result['id'] = test_seq_cand
result['Predicted'] = y_pred
result.to_csv("dados_lab_classificacao/resultado_lr_cv_10.csv", index=False)

Train
Acuracy:  0.9128837575439517
Precision:  0.939537640782454
Recall:  0.9611885991510006
AUC - P&R:  0.9931293649332831


C:\Users\laris\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
